#### Imports 

In [3]:
from time import gmtime, strftime
from sagemaker import image_uris
import sagemaker
import logging
import boto3
import json

#### Setup

In [4]:
logger = logging.getLogger('__name__')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

In [5]:
logger.info(f'Using SageMaker version: {sagemaker.__version__}')

Using SageMaker version: 2.49.0


In [6]:
region = sagemaker.Session().boto_region_name
role = sagemaker.get_execution_role()
boto3.setup_default_session(region_name=region)
boto_session = boto3.Session(region_name=region)
sagemaker_client = boto_session.client('sagemaker')
current_timestamp = strftime('%m-%d-%H-%M', gmtime())
logger.info(f'Region = {region}')

Region = us-east-1


## Create a real-time endpoint using SageMaker Model Registry

#### Re-create Model using Model Package ARN from Model Registry

**Note:** Can only create a Model in `Approved` state.

In [10]:
MODEL_PACKAGE_ARN = 'arn:aws:sagemaker:us-east-1:892313895307:model-package/bert-email-classifier/1'

In [11]:
model_name = f'bert-email-classifier-{current_timestamp}'

In [12]:
container = {'ModelPackageName': MODEL_PACKAGE_ARN}
response = sagemaker_client.create_model(ModelName=model_name, 
                                         ExecutionRoleArn=role, 
                                         Containers=[container])
response

{'ModelArn': 'arn:aws:sagemaker:us-east-1:892313895307:model/bert-email-classifier-08-09-20-12',
 'ResponseMetadata': {'RequestId': '938a8f60-09de-478e-a298-b8c86b1c669d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '938a8f60-09de-478e-a298-b8c86b1c669d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '95',
   'date': 'Mon, 09 Aug 2021 20:23:43 GMT'},
  'RetryAttempts': 0}}

### Create Endpoint Configuration

In [13]:
endpoint_config_name = model_name

In [14]:
response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant-1",
            "ModelName": model_name,
            "InstanceType": "ml.m5.xlarge",
            "InitialInstanceCount": 1
        }
    ]
)
endpoint_config_arn = response['EndpointConfigArn']
logger.info(f"Created EndpointConfig: {endpoint_config_arn}")

Created EndpointConfig: arn:aws:sagemaker:us-east-1:892313895307:endpoint-config/bert-email-classifier-08-09-20-12
